# Pipeline Machine Learning: Prediksi Rating Sepatu Pria

Notebook ini berisi pipeline lengkap untuk analisis dan pemodelan machine learning pada dataset sepatu pria. Langkah-langkah yang akan dilakukan meliputi:

1.  **Eksplorasi Data (EDA)**: Memahami distribusi dan korelasi data.
2.  **Preprocessing Data**: Membersihkan, melakukan imputasi, encoding, dan scaling.
3.  **Pembagian Dataset**: Memisahkan data menjadi set training dan testing.
4.  **Pelatihan Model**: Melatih minimal 3 algoritma (Regresi dan Klasifikasi).
5.  **Evaluasi Model**: Menganalisis performa dengan metrik yang sesuai (Classification Report & metrik regresi).
6.  **Visualisasi Performa**: Membandingkan kinerja antar model.
7.  **Hyperparameter Tuning**: Mencari parameter terbaik untuk model.
8.  **Cross-Validation**: Memvalidasi ketahanan (robustness) model.

--- 
### Langkah 1: Setup dan Import Library
Pertama, kita akan mengimpor semua library yang dibutuhkan untuk analisis ini.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, classification_report, accuracy_score

# Konfigurasi untuk visualisasi dan output
plt.style.use('seaborn-v0_8-whitegrid')
warnings.filterwarnings('ignore')

--- 
### Langkah 2: Memuat dan Membersihkan Data
Kita akan memuat dataset `MEN_SHOES.csv`, membersihkan kolom harga dari simbol mata uang, dan mengisi nilai yang hilang.

In [ ]:
try:
    df = pd.read_csv('MEN_SHOES.csv')
    print("Dataset berhasil dimuat.")
except FileNotFoundError:
    print("ERROR: File 'MEN_SHOES.csv' tidak ditemukan. Pastikan file berada di direktori yang sama.")

# Buat salinan untuk menjaga data asli
df_cleaned = df.copy()

# Hapus simbol mata uang '₹' dan koma ',' dari kolom harga
df_cleaned['Current_Price'] = df_cleaned['Current_Price'].str.replace('₹', '').str.replace(',', '').astype(float)

# Hapus koma ',' dari kolom penjualan
df_cleaned['How_Many_Sold'] = df_cleaned['How_Many_Sold'].str.replace(',', '').astype(float)

# Imputasi nilai yang hilang di kolom harga dengan median
median_price = df_cleaned['Current_Price'].median()
df_cleaned['Current_Price'].fillna(median_price, inplace=True)

print("\nData setelah dibersihkan:")
df_cleaned.head()

--- 
### Langkah 3: Exploratory Data Analysis (EDA)
Visualisasi data untuk memahami distribusi dan hubungan antar fitur.

In [ ]:
# 1. Histogram untuk fitur numerik
fig, axes = plt.subplots(1, 3, figsize=(20, 6))
sns.histplot(df_cleaned['Current_Price'], bins=30, kde=True, ax=axes[0])
axes[0].set_title('Distribusi Harga (Current_Price)')

sns.histplot(df_cleaned['How_Many_Sold'], bins=30, kde=True, ax=axes[1])
axes[1].set_title('Distribusi Jumlah Terjual (How_Many_Sold)')

sns.histplot(df_cleaned['RATING'], bins=20, kde=True, ax=axes[2])
axes[2].set_title('Distribusi Rating')
plt.tight_layout()
plt.show()

# 2. Boxplot untuk melihat outliers
fig, axes = plt.subplots(1, 3, figsize=(20, 6))
sns.boxplot(y=df_cleaned['Current_Price'], ax=axes[0])
axes[0].set_title('Boxplot Harga')

sns.boxplot(y=df_cleaned['How_Many_Sold'], ax=axes[1])
axes[1].set_title('Boxplot Jumlah Terjual')

sns.boxplot(y=df_cleaned['RATING'], ax=axes[2])
axes[2].set_title('Boxplot Rating')
plt.tight_layout()
plt.show()

# 3. Heatmap Korelasi
plt.figure(figsize=(10, 8))
numerical_cols = df_cleaned.select_dtypes(include=np.number)
correlation_matrix = numerical_cols.corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Heatmap Korelasi Antar Fitur Numerik')
plt.show()

--- 
### Langkah 4: Preprocessing dan Pembagian Dataset
Sekarang kita akan menyiapkan data untuk pemodelan:
1.  **Definisi Fitur (X) dan Target (y)**: Kita akan membuat dua target, satu untuk regresi (`y_reg`) dan satu untuk klasifikasi (`y_clf`).
2.  **Preprocessing Pipeline**: Membuat `ColumnTransformer` untuk melakukan scaling pada fitur numerik dan encoding pada fitur kategorikal. **Penting**: `sparse_output=False` ditambahkan untuk mengatasi error pada model `GaussianNB`.
3.  **Pembagian Data**: Membagi data menjadi 80% training dan 20% testing.

In [ ]:
# 1. Definisi Fitur (X) dan Target (y)
X = df_cleaned.drop(columns=['RATING', 'Product_details'])
y_reg = df_cleaned['RATING'] # Target untuk regresi

# Buat target untuk klasifikasi dengan mengubah rating menjadi kategori
y_clf = pd.cut(df_cleaned['RATING'], 
               bins=[0, 3.8, 4.0, 5], 
               labels=['Buruk', 'Rata-rata', 'Baik'], 
               include_lowest=True).fillna('Rata-rata')

# 2. Preprocessing Pipeline
numerical_features = X.select_dtypes(include='number').columns.tolist()
categorical_features = ['Brand_Name']

# **PENTING**: sparse_output=False untuk memastikan output dari preprocessor adalah dense array
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ],
    sparse_output=False
)

# 3. Pembagian Data
# Untuk Regresi
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X, y_reg, test_size=0.2, random_state=42)

# Untuk Klasifikasi
X_train_clf, X_test_clf, y_train_clf, y_test_clf = train_test_split(X, y_clf, test_size=0.2, random_state=42, stratify=y_clf)

print(f"Ukuran data training untuk klasifikasi: {X_train_clf.shape}")
print(f"Ukuran data testing untuk klasifikasi: {X_test_clf.shape}")

--- 
### Langkah 5 & 6: Pelatihan dan Evaluasi Model
Kita akan melatih semua model yang dipilih pada data training dan mengevaluasi performanya pada data testing.

#### 5.1. Model Regresi

In [ ]:
regression_models = {
    'Linear Regression': LinearRegression(),
    'Decision Tree Regressor': DecisionTreeRegressor(random_state=42),
    'Support Vector Regressor (SVR)': SVR()
}

print("--- EVALUASI MODEL REGRESI ---")

for name, model in regression_models.items():
    # Buat pipeline lengkap dengan preprocessor dan model
    pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('model', model)])
    pipeline.fit(X_train_reg, y_train_reg)
    y_pred = pipeline.predict(X_test_reg)

    # Evaluasi
    mae = mean_absolute_error(y_test_reg, y_pred)
    mse = mean_squared_error(y_test_reg, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test_reg, y_pred)

    print(f"\nModel: {name}")
    print(f"  Mean Absolute Error (MAE): {mae:.4f}")
    print(f"  Mean Squared Error (MSE): {mse:.4f}")
    print(f"  Root Mean Squared Error (RMSE): {rmse:.4f}")
    print(f"  R-squared (R²): {r2:.4f}")

#### 5.2. Model Klasifikasi

In [ ]:
classification_models = {
    'Decision Tree Classifier': DecisionTreeClassifier(random_state=42),
    'K-Nearest Neighbors (KNN)': KNeighborsClassifier(),
    'Gaussian Naive Bayes': GaussianNB()
}

print("--- LAPORAN KINERJA MODEL KLASIFIKASI ---")

for name, model in classification_models.items():
    pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('classifier', model)])
    pipeline.fit(X_train_clf, y_train_clf)
    y_pred = pipeline.predict(X_test_clf)
    
    accuracy = accuracy_score(y_test_clf, y_pred)
    
    print("\n" + "="*50)
    print(f"MODEL: {name}")
    print(f"AKURASI: {accuracy:.4f}")
    print("="*50)
    print(classification_report(y_test_clf, y_pred, zero_division=0))

--- 
### Langkah 7: Visualisasi Performa Model
Membuat diagram batang untuk membandingkan performa dari semua model secara visual.

In [ ]:
# Visualisasi akan dibuat pada langkah terakhir untuk membandingkan model dasar, setelah tuning, dan cross-validation.
# Untuk saat ini, kita bisa simpan hasilnya untuk perbandingan nanti.

# Namun, mari kita buat visualisasi awal untuk model dasar.
reg_perf = {}
for name, model in regression_models.items():
    pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('model', model)])
    pipeline.fit(X_train_reg, y_train_reg)
    y_pred = pipeline.predict(X_test_reg)
    reg_perf[name] = r2_score(y_test_reg, y_pred)

clf_perf = {}
for name, model in classification_models.items():
    pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('classifier', model)])
    pipeline.fit(X_train_clf, y_train_clf)
    y_pred = pipeline.predict(X_test_clf)
    clf_perf[name] = accuracy_score(y_test_clf, y_pred)
    

fig, ax = plt.subplots(1, 2, figsize=(18, 7))
pd.Series(reg_perf).sort_values().plot(kind='barh', ax=ax[0], title='Perbandingan Model Regresi (R-squared)')
ax[0].set_xlabel('R-squared Score')

pd.Series(clf_perf).sort_values().plot(kind='barh', ax=ax[1], title='Perbandingan Model Klasifikasi (Accuracy)')
ax[1].set_xlabel('Accuracy Score')

plt.tight_layout()
plt.show()

--- 
### Langkah 8: Hyperparameter Tuning
Kita akan mengambil model klasifikasi dengan performa terbaik (berdasarkan akurasi) dan mencoba mencari kombinasi hyperparameter yang lebih optimal menggunakan `GridSearchCV`.

In [ ]:
# Cari nama model klasifikasi dengan akurasi tertinggi
best_clf_name = max(clf_perf, key=clf_perf.get)
print(f"Model klasifikasi terbaik adalah: {best_clf_name} dengan akurasi awal {clf_perf[best_clf_name]:.4f}\n")

# Definisikan pipeline dan parameter grid untuk model terbaik
best_model_pipeline = Pipeline(steps=[('preprocessor', preprocessor), 
                                      ('classifier', classification_models[best_clf_name])])

param_grid = {}
if 'Decision Tree' in best_clf_name:
    param_grid = {
        'classifier__max_depth': [5, 10, 15, None],
        'classifier__min_samples_leaf': [1, 2, 4],
        'classifier__criterion': ['gini', 'entropy']
    }
elif 'KNN' in best_clf_name:
    param_grid = {
        'classifier__n_neighbors': [3, 5, 7, 9],
        'classifier__weights': ['uniform', 'distance']
    }

# Lakukan Grid Search jika modelnya bukan Gaussian NB
if param_grid:
    grid_search = GridSearchCV(best_model_pipeline, param_grid, cv=5, n_jobs=-1, scoring='accuracy')
    grid_search.fit(X_train_clf, y_train_clf)
    
    print(f"Tuning Model: {best_clf_name}")
    print(f"Parameter Terbaik: {grid_search.best_params_}")
    print(f"Akurasi Terbaik (setelah tuning): {grid_search.best_score_:.4f}")
    
    # Simpan model terbaik setelah tuning
    best_model_after_tuning = grid_search.best_estimator_
else:
    print(f"{best_clf_name} tidak memiliki hyperparameter umum untuk dituning dalam skenario ini.")
    best_model_after_tuning = best_model_pipeline.fit(X_train_clf, y_train_clf)

--- 
### Langkah 9: Cross-Validation
Terakhir, kita akan menerapkan cross-validation pada model terbaik untuk memastikan performanya stabil dan tidak hanya kebetulan bagus pada satu pembagian data saja (robustness).

In [ ]:
print("--- CROSS-VALIDATION UNTUK ROBUSTNESS ---")

# CV untuk model regresi terbaik (berdasarkan R2 awal)
best_reg_name = max(reg_perf, key=reg_perf.get)
pipeline_reg_cv = Pipeline(steps=[('preprocessor', preprocessor), ('model', regression_models[best_reg_name])])
cv_scores_reg = cross_val_score(pipeline_reg_cv, X, y_reg, cv=5, scoring='r2')

print(f"Cross-Validation untuk Regresi: {best_reg_name}")
print(f"  Skor R2 (per fold): {np.round(cv_scores_reg, 4)}")
print(f"  Rata-rata R2: {cv_scores_reg.mean():.4f} (+/- {cv_scores_reg.std():.4f})\n")

# CV untuk model klasifikasi terbaik setelah tuning
cv_scores_clf = cross_val_score(best_model_after_tuning, X, y_clf, cv=5, scoring='accuracy')
print(f"Cross-Validation untuk Klasifikasi: {best_clf_name} (Setelah Tuning)")
print(f"  Skor Akurasi (per fold): {np.round(cv_scores_clf, 4)}")
print(f"  Rata-rata Akurasi: {cv_scores_clf.mean():.4f} (+/- {cv_scores_clf.std():.4f})")